In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

def get_economic_cycle_indicators():
    """
    Coleta indicadores econômicos para determinar o ciclo vigente
    """
    # Período de análise (últimos 2 anos)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=730)
    
    indicators = {}
    
    try:
        print("Coletando dados dos Treasuries...")
        # 1. Curva de Juros (10Y-2Y Treasury Spread)
        treasury_10y = yf.download("^TNX", start=start_date, end=end_date, progress=False)
        treasury_2y = yf.download("^IRX", start=start_date, end=end_date, progress=False)
        
        if not treasury_10y.empty and not treasury_2y.empty and 'Close' in treasury_10y.columns and 'Close' in treasury_2y.columns:
            # Pegar apenas os valores válidos (não-NaN)
            t10_clean = treasury_10y['Close'].dropna()
            t2_clean = treasury_2y['Close'].dropna()
            
            if len(t10_clean) > 0 and len(t2_clean) > 0:
                # Usar o último valor disponível
                yield_10y = float(t10_clean.iloc[-1])
                yield_2y = float(t2_clean.iloc[-1])
                
                indicators['yield_curve_current'] = yield_10y - yield_2y
                indicators['yield_curve_trend'] = 'Invertida' if indicators['yield_curve_current'] < 0 else 'Normal'
                print(f"✅ Treasuries coletados: 10Y={yield_10y:.2f}%, 2Y={yield_2y:.2f}%")
            else:
                print("⚠️ Dados de Treasury sem valores válidos")
        else:
            print("⚠️ Erro ao baixar dados de Treasury")
        
        # 2. S&P 500 Performance e Volatilidade
        sp500 = yf.download("^GSPC", start=start_date, end=end_date, progress=False)
        if not sp500.empty:
            sp500_returns = sp500['Close'].pct_change().dropna()
            
            indicators['sp500_ytd_return'] = float(((sp500['Close'].iloc[-1] / sp500['Close'].iloc[0]) - 1) * 100)
            indicators['sp500_volatility'] = float(sp500_returns.std() * np.sqrt(252) * 100)  # Anualizada
        
        # 3. VIX (Fear Index)
        vix = yf.download("^VIX", start=start_date, end=end_date, progress=False)['Close']
        if not vix.empty:
            indicators['vix_current'] = float(vix.iloc[-1])
            indicators['vix_avg_3m'] = float(vix.tail(63).mean()) if len(vix) >= 63 else np.nan
        
        # 4. Dollar Index (DXY)
        dxy = yf.download("DX-Y.NYB", start=start_date, end=end_date, progress=False)['Close']
        if not dxy.empty:
            indicators['dxy_current'] = float(dxy.iloc[-1])
            indicators['dxy_change_3m'] = float(((dxy.iloc[-1] / dxy.iloc[-63]) - 1) * 100) if len(dxy) >= 63 else np.nan
        
        # 5. Commodities (Oil - WTI)
        oil = yf.download("CL=F", start=start_date, end=end_date, progress=False)['Close']
        if not oil.empty:
            indicators['oil_current'] = float(oil.iloc[-1])
            indicators['oil_change_3m'] = float(((oil.iloc[-1] / oil.iloc[-63]) - 1) * 100) if len(oil) >= 63 else np.nan
        
        # 6. High Yield vs Treasury Spread
        hy_bond = yf.download("HYG", start=start_date, end=end_date, progress=False)['Close']
        treasury_etf = yf.download("IEF", start=start_date, end=end_date, progress=False)['Close']
        
        if not hy_bond.empty and not treasury_etf.empty:
            hy_returns = hy_bond.pct_change().dropna()
            treasury_returns = treasury_etf.pct_change().dropna()
            
            # Correlação como proxy para spread de crédito
            if len(hy_returns) > 30 and len(treasury_returns) > 30:
                common_dates = hy_returns.index.intersection(treasury_returns.index)
                if len(common_dates) > 30:
                    correlation = hy_returns[common_dates].corr(treasury_returns[common_dates])
                    indicators['credit_risk_signal'] = 'Alto' if correlation < -0.3 else 'Baixo'
        
    except Exception as e:
        print(f"Erro ao coletar dados: {e}")
    
    return indicators

def determine_economic_cycle(indicators):
    """
    Determina o ciclo econômico baseado nos indicadores
    """
    score = 0
    signals = []
    
    # Análise da Curva de Juros
    if 'yield_curve_current' in indicators:
        if indicators['yield_curve_current'] < -0.5:
            score -= 2
            signals.append("⚠️ Curva de juros invertida (recessão)")
        elif indicators['yield_curve_current'] < 0:
            score -= 1
            signals.append("⚠️ Curva de juros levemente invertida")
        elif indicators['yield_curve_current'] > 2:
            score += 1
            signals.append("✅ Curva de juros normal e íngreme")
    
    # Análise do VIX
    if 'vix_current' in indicators:
        if indicators['vix_current'] > 30:
            score -= 2
            signals.append("⚠️ VIX alto - medo no mercado")
        elif indicators['vix_current'] > 20:
            score -= 1
            signals.append("⚠️ VIX moderado - alguma tensão")
        else:
            score += 1
            signals.append("✅ VIX baixo - mercado tranquilo")
    
    # Análise do S&P 500
    if 'sp500_ytd_return' in indicators:
        if indicators['sp500_ytd_return'] < -15:
            score -= 2
            signals.append("⚠️ S&P 500 em bear market")
        elif indicators['sp500_ytd_return'] < -5:
            score -= 1
            signals.append("⚠️ S&P 500 com performance negativa")
        elif indicators['sp500_ytd_return'] > 15:
            score += 1
            signals.append("✅ S&P 500 com forte performance")
    
    # Análise de Volatilidade
    if 'sp500_volatility' in indicators:
        if indicators['sp500_volatility'] > 25:
            score -= 1
            signals.append("⚠️ Alta volatilidade no mercado")
        elif indicators['sp500_volatility'] < 15:
            score += 1
            signals.append("✅ Baixa volatilidade")
    
    # Determinação do ciclo
    if score >= 2:
        cycle = "🟢 EXPANSÃO"
        description = "Economia em crescimento, mercados otimistas"
        recommendations = [
            "Foco em growth stocks e tecnologia",
            "Reduzir posição em bonds de longo prazo",
            "Considerar REITs e commodities",
            "Aumentar exposição a small caps"
        ]
    elif score >= 0:
        cycle = "🟡 EXPANSÃO TARDIA"
        description = "Crescimento desacelerando, inflação possível"
        recommendations = [
            "Diversificar entre growth e value",
            "Considerar TIPS (inflation-protected securities)",
            "Manter posição defensiva em utilities",
            "Reduzir duration em bonds"
        ]
    elif score >= -2:
        cycle = "🟠 DESACELERAÇÃO"
        description = "Economia desacelerando, mercados cautelosos"
        recommendations = [
            "Migrar para value stocks e dividendos",
            "Aumentar posição em bonds de alta qualidade",
            "Considerar setores defensivos (utilities, consumer staples)",
            "Reduzir exposição a growth e tecnologia"
        ]
    else:
        cycle = "🔴 RECESSÃO"
        description = "Contração econômica, mercados em stress"
        recommendations = [
            "Posição defensiva: cash e treasuries",
            "Foco em dividend aristocrats",
            "Evitar high yield e junk bonds",
            "Considerar inverse ETFs para hedge"
        ]
    
    return {
        'cycle': cycle,
        'description': description,
        'score': score,
        'signals': signals,
        'recommendations': recommendations
    }

def display_cycle_analysis():
    """
    Executa a análise completa e exibe os resultados
    """
    print("🔍 ANALISANDO CICLO ECONÔMICO DO MERCADO AMERICANO...")
    print("=" * 60)
    
    # Coleta indicadores
    indicators = get_economic_cycle_indicators()
    
    # Exibe indicadores principais
    print("\n📊 INDICADORES ECONÔMICOS ATUAIS:")
    print("-" * 40)
    
    if 'yield_curve_current' in indicators and not np.isnan(indicators['yield_curve_current']):
        print(f"Curva de Juros (10Y-2Y): {indicators['yield_curve_current']:.2f}% ({indicators['yield_curve_trend']})")
    
    if 'vix_current' in indicators and not np.isnan(indicators['vix_current']):
        print(f"VIX (Índice do Medo): {indicators['vix_current']:.1f}")
    
    if 'sp500_ytd_return' in indicators and not np.isnan(indicators['sp500_ytd_return']):
        print(f"S&P 500 YTD: {indicators['sp500_ytd_return']:.1f}%")
    
    if 'sp500_volatility' in indicators and not np.isnan(indicators['sp500_volatility']):
        print(f"Volatilidade S&P 500: {indicators['sp500_volatility']:.1f}%")
    
    if 'dxy_current' in indicators and not np.isnan(indicators['dxy_current']):
        print(f"Índice do Dólar (DXY): {indicators['dxy_current']:.1f}")
    
    if 'oil_current' in indicators and not np.isnan(indicators['oil_current']):
        print(f"Petróleo WTI: ${indicators['oil_current']:.1f}")
    
    # Determina ciclo
    cycle_analysis = determine_economic_cycle(indicators)
    
    print(f"\n🎯 CICLO ECONÔMICO IDENTIFICADO:")
    print("-" * 40)
    print(f"Status: {cycle_analysis['cycle']}")
    print(f"Descrição: {cycle_analysis['description']}")
    print(f"Score de Confiança: {cycle_analysis['score']}")
    
    print(f"\n🔍 SINAIS IDENTIFICADOS:")
    print("-" * 40)
    for signal in cycle_analysis['signals']:
        print(f"  {signal}")
    
    print(f"\n💡 RECOMENDAÇÕES PARA CARTEIRA:")
    print("-" * 40)
    for i, rec in enumerate(cycle_analysis['recommendations'], 1):
        print(f"  {i}. {rec}")
    
    print("\n" + "=" * 60)
    print("✅ Análise concluída! Use essas informações para ajustar sua estratégia de alocação.")
    
    return cycle_analysis, indicators

# Executar análise
if __name__ == "__main__":
    cycle_result, raw_indicators = display_cycle_analysis()

🔍 ANALISANDO CICLO ECONÔMICO DO MERCADO AMERICANO...
Coletando dados dos Treasuries...
✅ Treasuries coletados: 10Y=4.20%, 2Y=4.16%
Erro ao coletar dados: "None of [DatetimeIndex(['2023-08-08', '2023-08-09', '2023-08-10', '2023-08-11',\n               '2023-08-14', '2023-08-15', '2023-08-16', '2023-08-17',\n               '2023-08-18', '2023-08-21',\n               ...\n               '2025-07-22', '2025-07-23', '2025-07-24', '2025-07-25',\n               '2025-07-28', '2025-07-29', '2025-07-30', '2025-07-31',\n               '2025-08-01', '2025-08-04'],\n              dtype='datetime64[ns]', name='Date', length=499, freq=None)] are in the [columns]"

📊 INDICADORES ECONÔMICOS ATUAIS:
----------------------------------------
Curva de Juros (10Y-2Y): 0.04% (Normal)
VIX (Índice do Medo): 17.9
S&P 500 YTD: 39.8%
Volatilidade S&P 500: 16.1%
Índice do Dólar (DXY): 98.8
Petróleo WTI: $66.3

🎯 CICLO ECONÔMICO IDENTIFICADO:
----------------------------------------
Status: 🟢 EXPANSÃO
Descrição: E

In [ ]:
# Bloco 1: Instalação das bibliotecas necessárias
!pip install pandas requests schedule openpyxl

# Importação das bibliotecas
import pandas as pd
import requests
import schedule
import time
from datetime import datetime

print("Bibliotecas instaladas e importadas com sucesso!")


🎯 CONSTRUINDO PORTFÓLIO PARA CICLO: EXPANSÃO
📊 Analisando ETFs...
Erro ao analisar QQQ: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Erro ao analisar XLK: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Erro ao analisar ARKK: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Erro ao analisar SOXX: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Erro ao analisar VTI: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Erro ao analisar IWM: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Erro ao analisar VUG: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().
Erro ao analisar XLF: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all

In [5]:
#GPT

import os
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
from databento import Historical, Dataset

# 1) Carregar variável de ambiente
load_dotenv()
api_key = os.getenv("DB_API_KEY")
if not api_key:
    raise ValueError("API key não encontrada no .env!")

# 2) Inicializar o cliente
client = Historical(api_key=api_key)

# 3) Defina os ETFs de setor (como proxy)
SECTOR_ETFS = {
    "Consumer Discretionary": "XLY",
    "Consumer Staples":       "XLP",
    "Energy":                 "XLE",
    "Financials":             "XLF",
    "Health Care":            "XLV",
    "Industrials":            "XLI",
    "Materials":              "XLB",
    "Technology":             "XLK",
    "Utilities":              "XLU"
}

# 4) Baixar dados históricos para cada ETF
def get_sector_data(etfs, start_date, end_date):
    df_all = pd.DataFrame()
    for sector, symbol in etfs.items():
        print(f"Baixando dados de {symbol} ({sector})...")
        try:
            data = client.timeseries.get_range(
                dataset=Dataset.ETF_NBBO,  # Pode ser Dataset.IEX se preferir
                symbols=symbol,
                schema="ohlcv-1m",
                start=start_date,
                end=end_date,
                limit=1000
            )

            df = data.to_df()
            df["symbol"] = symbol
            df["sector"] = sector
            df["date"] = pd.to_datetime(df["ts_event"], unit="s").dt.date
            df_day = df.groupby("date")[["open", "high", "low", "close", "volume"]].mean().reset_index()
            df_day["sector"] = sector
            df_all = pd.concat([df_all, df_day], ignore_index=True)

        except Exception as e:
            print(f"Erro ao baixar {symbol}: {e}")

    return df_all

# 5) Definir intervalo
end_date = datetime.utcnow().date()
start_date = end_date - timedelta(days=30)

# 6) Obter os dados
dados_setoriais = get_sector_data(SECTOR_ETFS, start_date.isoformat(), end_date.isoformat())

# 7) Exibir resultado
print(dados_setoriais.head())


TypeError: Historical.__init__() got an unexpected keyword argument 'api_key'

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import requests
import yfinance as yf
import databento as db  # Certifique-se de ter instalado: pip install databento

# Configurações iniciais
plt.style.use('ggplot')
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# Configuração das APIs (substitua com suas chaves)
DATABENTO_API_KEY = "db-jpmKnEhu74j5tSBkDCma9WMuuu3sT "
FMP_API_KEY = "wzKZBImlIr1BLlc11ZXoh5DEkJdhkfnQ"

def get_databento_historical(ticker, start_date, end_date):
    """Obtém dados históricos da DataBento"""
    try:
        client = db.Historical(DATABENTO_API_KEY)
        
        # Converter símbolos para o formato DataBento
        db_symbol = {
            'SPY': 'SPY.AMEX',
            'TLT': 'TLT.NASDAQ',
            'GLD': 'GLD.NYSE',
            'IEF': 'IEF.NASDAQ',
            'QQQ': 'QQQ.NASDAQ',
            'DIA': 'DIA.NYSE',
            'SHY': 'SHY.NYSE'
        }.get(ticker, f"{ticker}.NYSE")  # Padrão para NYSE
        
        data = client.timeseries.get_range(
            dataset='XNAS.ITCH',  # Ajuste conforme necessário
            symbols=[db_symbol],
            schema='ohlcv-1d',
            start=start_date,
            end=end_date,
            stype_in='smart',
        )
        
        if not data.empty:
            df = data.reset_index()
            df['date'] = pd.to_datetime(df['ts_event'])
            df.set_index('date', inplace=True)
            return df['close']  # Usando preço de fechamento ajustado
        
        print(f"DataBento: Dados insuficientes para {ticker}")
        return None
    
    except Exception as e:
        print(f"Erro na DataBento para {ticker}: {str(e)}")
        return None

def get_fmp_historical(ticker, start_date, end_date):
    """Fallback para FMP"""
    try:
        url = f"https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?" \
              f"from={start_date}&to={end_date}&apikey={FMP_API_KEY}"
        
        response = requests.get(url)
        data = response.json()
        
        if 'historical' in data and len(data['historical']) > 0:
            df = pd.DataFrame(data['historical'])
            df['date'] = pd.to_datetime(df['date'])
            df.set_index('date', inplace=True)
            return df['adjClose']
        
        print(f"FMP: Dados insuficientes para {ticker}")
        return None
    
    except Exception as e:
        print(f"Erro na FMP para {ticker}: {str(e)}")
        return None

def get_yfinance_historical(ticker, start_date, end_date):
    """Último fallback para Yahoo Finance"""
    try:
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)
        if not data.empty:
            return data['Adj Close'] if 'Adj Close' in data.columns else data['Close']
        return None
    except Exception as e:
        print(f"Erro no Yahoo Finance para {ticker}: {str(e)}")
        return None

def get_market_data(tickers, start_date, end_date):
    """Obtém dados hierarquicamente: DataBento > FMP > Yahoo Finance"""
    market_data = {}
    
    for ticker in tickers:
        print(f"\nObtendo {ticker}:")
        
        # 1. Tentar DataBento primeiro
        print("  Tentando DataBento...", end=' ')
        data = get_databento_historical(ticker, start_date, end_date)
        source = "DataBento" if data is not None else None
        
        # 2. Fallback para FMP
        if data is None:
            print("\n  Tentando FMP...", end=' ')
            data = get_fmp_historical(ticker, start_date, end_date)
            source = "FMP" if data is not None else None
        
        # 3. Último fallback para Yahoo Finance
        if data is None:
            print("\n  Tentando Yahoo Finance...", end=' ')
            data = get_yfinance_historical(ticker, start_date, end_date)
            source = "YFinance" if data is not None else None
        
        if data is not None:
            # Verificação de qualidade dos dados
            if isinstance(data.index, pd.DatetimeIndex) and len(data) > 10:
                market_data[ticker] = data
                print(f"✅ (Fonte: {source}, {len(data)} períodos)")
            else:
                print("❌ Dados insuficientes ou formato inválido")
        else:
            print("❌ Todas as fontes falharam")
    
    # Combinação final dos dados
    if market_data:
        # Encontrar o índice de data mais completo
        all_dates = pd.concat([pd.Series(idx) for idx in market_data.values()], axis=0).unique()
        reference_index = pd.to_datetime(sorted(all_dates))
        
        # Criar DataFrame alinhado
        aligned_data = pd.DataFrame(index=reference_index)
        for ticker, series in market_data.items():
            aligned_data[ticker] = series.reindex(reference_index).ffill().bfill()
        
        return aligned_data.dropna(how='all')
    return None

def analyze_market_cycle():
    """Analisa o ciclo atual do mercado com dados profissionais"""
    assets = {
        'SPY': 'S&P 500',
        'TLT': 'Títulos Longos',
        'GLD': 'Ouro',
        'IEF': 'Títulos Médios',
        'QQQ': 'Nasdaq',
        'DIA': 'Dow Jones',
        'SHY': 'Títulos Curtos'
    }
    
    end_date = datetime.now().strftime('%Y-%m-%d')
    start_date = (datetime.now() - timedelta(days=5*365)).strftime('%Y-%m-%d')
    
    print("\n🔍 Iniciando análise profissional de ciclo de mercado")
    print(f"Período analisado: {start_date} a {end_date}")
    
    market_data = get_market_data(assets.keys(), start_date, end_date)
    
    if market_data is None or market_data.empty:
        print("\n⚠️ Erro crítico: Não foi possível obter dados suficientes")
        return None
    
    # Verificação final
    print("\n✅ Dados obtidos com sucesso:")
    print(f"Ativos disponíveis: {list(market_data.columns)}")
    print(f"Total de períodos: {len(market_data)}")
    print(f"Primeira data: {market_data.index[0].date()}")
    print(f"Última data: {market_data.index[-1].date()}")
    
    # Análise de desempenho relativo
    normalized = (market_data / market_data.iloc[0] * 100)
    
    # Plotagem profissional
    plt.figure(figsize=(14, 7))
    for col in normalized.columns:
        plt.plot(normalized.index, normalized[col], label=f"{col} ({assets[col]})", linewidth=2)
    
    plt.title('Desempenho Relativo Normalizado (Base 100)', fontsize=14)
    plt.ylabel('Retorno Normalizado', fontsize=12)
    plt.xlabel('Data', fontsize=12)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()
    
    # Análise de momentum (6 meses)
    momentum = market_data.pct_change(126).iloc[-1].sort_values(ascending=False)
    
    # DataFrame formatado
    momentum_df = pd.DataFrame({
        'Ativo': momentum.index,
        'Retorno 6M': momentum.values,
        'Descrição': [assets[t] for t in momentum.index]
    })
    momentum_df['Retorno 6M'] = momentum_df['Retorno 6M'].apply(lambda x: f"{x:.1%}")
    
    print("\n📈 Análise de Momentum (6 meses):")
    print(momentum_df.to_markdown(index=False, tablefmt="grid"))
    
    # Diagnóstico avançado do ciclo
    if len(momentum) > 0:
        top_asset = momentum.index[0]
        top_return = momentum.iloc[0]
        
        if top_return > 0.15:
            if top_asset in ['SPY', 'QQQ', 'DIA']:
                cycle = "🟢 MERCADO EM ALTA (RISK-ON)"
                suggestion = """Estratégia recomendada:
                - Aumentar alocação em ações growth
                - Considerar alavancagem moderada
                - Reduzir duration de títulos"""
            elif top_asset in ['TLT', 'IEF', 'SHY']:
                cycle = "🟡 MERCADO DEFENSIVO"
                suggestion = """Estratégia recomendada:
                - Aumentar qualidade de crédito
                - Foco em ações defensivas (utilities, consumer staples)
                - Aumentar duration"""
            elif top_asset == 'GLD':
                cycle = "🔴 AMBIENTE INFLACIONÁRIO"
                suggestion = """Estratégia recomendada:
                - Aumentar exposição a commodities
                - Considerar TIPS e ativos reais
                - Reduzir duration nominal"""
        else:
            cycle = "🔵 MERCADO EM CORREÇÃO"
            suggestion = """Estratégia recomendada:
            - Aumentar liquidez
            - Considerar estratégias de hedge
            - Manter disciplina de stop-loss"""
        
        print(f"\n🔍 DIAGNÓSTICO DO CICLO: {cycle}")
        print(suggestion)
    
    return market_data

# Execução principal
if __name__ == "__main__":
    print("Iniciando análise profissional de mercado...")
    final_data = analyze_market_cycle()
    
    if final_data is not None:
        print("\n💼 Análise concluída com dados profissionais")
        print("Dados disponíveis na variável 'final_data'")
        print("\nPróximos passos sugeridos:")
        print("- Analisar correlações entre ativos")
        print("- Otimizar alocações por ciclo")
        print("- Implementar regras de rebalanceamento")
    else:
        print("\n❌ A análise não pôde ser concluída")

Iniciando análise profissional de mercado...

🔍 Iniciando análise profissional de ciclo de mercado
Período analisado: 2020-08-05 a 2025-08-04

Obtendo SPY:
  Tentando DataBento... Erro na DataBento para SPY: 400 auth_invalid_username_in_basic_auth
Invalid username in Basic auth ('db-jpmKnEhu74j5tSBkDCma9WMuuu3sT ').
documentation: https://databento.com/docs

  Tentando FMP... ✅ (Fonte: FMP, 1255 períodos)

Obtendo TLT:
  Tentando DataBento... Erro na DataBento para TLT: 400 auth_invalid_username_in_basic_auth
Invalid username in Basic auth ('db-jpmKnEhu74j5tSBkDCma9WMuuu3sT ').
documentation: https://databento.com/docs

  Tentando FMP... ✅ (Fonte: FMP, 1255 períodos)

Obtendo GLD:
  Tentando DataBento... Erro na DataBento para GLD: 400 auth_invalid_username_in_basic_auth
Invalid username in Basic auth ('db-jpmKnEhu74j5tSBkDCma9WMuuu3sT ').
documentation: https://databento.com/docs

  Tentando FMP... ✅ (Fonte: FMP, 1255 períodos)

Obtendo IEF:
  Tentando DataBento... Erro na DataBento p